In [1]:
%autosave 3600

Autosaving every 3600 seconds


In [2]:
import bel_konzept as bk
import pandas as pd
import numpy as np

Dataframes für die Wohnung aufstellen und in Zu- und Ablufträume separieren:

In [4]:
df_whg = pd.DataFrame(
    [('Schlafen',13.47,'zu'),
     ('Bad',9.06,'ab',45),
     ('Abstellraum',2.57,'ab',25),
     ('Kochen',7.16,'ab',45),
     ('Wohnen',26.98,'zu'),
     ('Zimmer_2',13.96,'zu'),
     ('Zimmer_1',11.89,'zu'),
     ('WC',1.45,'ab',25),
     ('Flur',9.07,'--')
    ],
    columns=['Raum','Flaeche','Typ','dV_min']
)
df_whg

,Raum,Flaeche,Typ,dV_min
0,Schlafen,13.47,zu,NaN
1,Bad,9.06,ab,45.0
2,Abstellraum,2.57,ab,25.0
3,Kochen,7.16,ab,45.0
4,Wohnen,26.98,zu,NaN
5,Zimmer_2,13.96,zu,NaN
6,Zimmer_1,11.89,zu,NaN
7,WC,1.45,ab,25.0
8,Flur,9.07,--,NaN


In [6]:
df_zu = df_whg[df_whg.Typ=='zu'].copy()
df_zu.drop(columns=['dV_min'],inplace=True)
print('Zulufträume')
display(df_zu)

Zulufträume


,Raum,Flaeche,Typ
0,Schlafen,13.47,zu
4,Wohnen,26.98,zu
5,Zimmer_2,13.96,zu
6,Zimmer_1,11.89,zu


In [7]:
df_ab = df_whg[df_whg.Typ=='ab'].copy()
print('Ablufträume')
display(df_ab)

Ablufträume


,Raum,Flaeche,Typ,dV_min
1,Bad,9.06,ab,45.0
2,Abstellraum,2.57,ab,25.0
3,Kochen,7.16,ab,45.0
7,WC,1.45,ab,25.0


Zuluftvolumenstrom, Abluftvolumenstrom und Infiltrationsvolumenstrom ermitteln:

In [8]:
# Nutzfläche
A_N = df_whg.Flaeche.sum() # in m**2

# erforderlicher Zuluftvolumenstrom
dV_ges_zu = bk.dV_ges_zu(A_N=A_N) # in m**3/h
dV_ges_zu # in m**3/h

120.8102279

In [9]:
dV_ges_ab = df_ab.dV_min.sum()
dV_ges_ab # in m**3/h

140.0

Erforderlicher Gesamtvolumenstrom

In [11]:
dV_ges = max(dV_ges_zu,dV_ges_ab)
dV_ges # m**3/h

140.0

Infiltrationsvolumenstrom:

In [10]:
f_inf = 0.4 # Mehrgeschossig, Installationsschacht
n_50 = 1 # h**(-1)
h = 2.50 # m
delta_p = 5 # Mehrgeschossig, Windschwach

dV_inf = bk.dV_inf(f_inf,A_N,h,n_50,delta_p)
dV_inf # Infiltrationsvolumenstrom in m**3/h

20.58274616569973

Erforderlicher Lüftungsvolumenstrom am Ventilator

In [12]:
dV_L = dV_ges - dV_inf
dV_L # m**3/h

119.41725383430027

Verteilung der Zuluft auf die Zulufträume:

In [13]:
idx_zu = df_zu.index

# Faktoren
df_zu['f'] = pd.DataFrame([2,3,2,2],index=idx_zu)
df_zu['dV'] = df_zu.f/df_zu.f.sum()*dV_L

v = 2 # Zuluftgeschwindigkeit in m/s
hour = 3600 # 1 Stunde in Sekunden

df_zu['d_min'] = np.sqrt(4*df_zu.dV/(hour*np.pi*v))
df_zu['d_g'] = df_zu.apply(lambda x: bk.norm_durchmesser(1000*x.d_min)/1000,axis=1)
df_zu['v_g'] = (df_zu.d_min/df_zu.d_g)**2*v

rho = 1.2 # kg/m**3
df_zu['p_dyn'] = rho/2*(df_zu.v_g)**2
df_zu

,Raum,Flaeche,Typ,f,dV,d_min,d_g,v_g,p_dyn
0,Schlafen,13.47,zu,2,26.537168,0.068504,0.08,1.466500,1.290374
4,Wohnen,26.98,zu,3,39.805751,0.083900,0.09,1.738075,1.812542
5,Zimmer_2,13.96,zu,2,26.537168,0.068504,0.08,1.466500,1.290374
6,Zimmer_1,11.89,zu,2,26.537168,0.068504,0.08,1.466500,1.290374


Verteilung der Abluft auf die Ablufträume

In [14]:
idx_ab = df_ab.index

df_ab['dV'] = df_ab.dV_min/df_ab.dV_min.sum()*dV_L

df_ab['d_min'] = np.sqrt(4*df_ab.dV/(hour*np.pi*v))
df_ab['d_g'] = df_ab.apply(lambda x: bk.norm_durchmesser(1000*x.d_min)/1000,axis=1)
df_ab['v_g'] = (df_ab.d_min/df_ab.d_g)**2*v
df_ab['p_dyn'] = rho/2*(df_ab.v_g)**2

df_ab

,Raum,Flaeche,Typ,dV_min,dV,d_min,d_g,v_g,p_dyn
1,Bad,9.06,ab,45.0,38.384117,0.082388,0.09,1.676001,1.685387
2,Abstellraum,2.57,ab,25.0,21.324510,0.061408,0.08,1.178438,0.833230
3,Kochen,7.16,ab,45.0,38.384117,0.082388,0.09,1.676001,1.685387
7,WC,1.45,ab,25.0,21.324510,0.061408,0.08,1.178438,0.833230
